In [1]:
import os
import cv2
import time
import numpy as np
import argparse
import tensorflow as tf
import matplotlib.pyplot as plt

from src.data import extract_roi
from src.utils import cv_img_result

from imageai.Detection.Custom import CustomObjectDetection

In [2]:
D_MODEL_PATH = f"./data/images/box/models/detection_model-ex-003--loss-0013.547.h5"
JSON_FILE_PATH = f"./data/images/box/json/detection_config.json"
C_MODEL_PATH = f"./data/models/mobilnet/mobilenet.h5"
OUTPUT_FILE_NAME = f"tmp.jpg"

In [3]:
detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()

In [4]:
def model_load(detection_model_path,
               json_path,
               cls_model_path,):
    
    detector = CustomObjectDetection()
    detector.setModelTypeAsYOLOv3()
    
    detector.setModelPath(detection_model_path) 
    detector.setJsonPath(json_path)
    detector.loadModel()
    
    model = tf.keras.models.load_model(cls_model_path)
    
    return detector, model


In [5]:
json_path = JSON_FILE_PATH#args.json
detection_model_path = D_MODEL_PATH#args.detection_model
cls_model_path = C_MODEL_PATH#args.class_model
output_file = OUTPUT_FILE_NAME#args.output_file

detector, model = model_load(detection_model_path,
                                json_path,
                                cls_model_path)

print (f"Sucessfully loaded models")

Sucessfully loaded models


In [6]:
cap = cv2.VideoCapture(0)

while(True):

    ret, frame = cap.read()

    _, detections = detector.detectObjectsFromImage(input_image=frame,
                                                        input_type='array',
                                                        output_type='array')

    if len(detections)>0:
        roi_list = extract_roi(frame, detections)
        # Classify the object
        pred = model.predict(np.array(roi_list))

        frame, result_class = cv_img_result(frame, pred, detections)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
        

/Users/1112951/miniconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "
/Users/1112951/miniconda3/lib/python3.7/site-packages/imageai/Detection/Custom/__init__.py:1248: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(-x))


KeyboardInterrupt: 

In [7]:
cap.release()
cv2.destroyAllWindows()